# Capítulo III: Chains

### Ejemplo inicial en el uso de cadenas

Cada vez que se desee trabajar con una cadena, al menos, debe estar definido un prompt template e instanciado algún modelo de lenguaje

In [1]:
from dotenv import load_dotenv
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
)

In [2]:
load_dotenv()

True

In [3]:
human_message_prompt = HumanMessagePromptTemplate(
        prompt=PromptTemplate(
            template="Cual es una buena actividad para realizar en {temporada}?",
            input_variables=["temporada"],
        )
    )

In [4]:
chat_prompt_template = ChatPromptTemplate.from_messages([human_message_prompt])
chat = ChatOpenAI(temperature=0.9)
chain = LLMChain(llm=chat, prompt=chat_prompt_template)

In [5]:
print(chain.run("primavera"))

Una buena actividad para realizar en primavera es hacer un picnic al aire libre. Puedes elegir un parque o un jardín bonito, llevar una manta, comida y bebidas, y disfrutar de un día soleado rodeado de naturaleza. Además, puedes aprovechar para jugar diferentes juegos al aire libre, como lanzar una frisbee o practicar deportes en equipo.


In [6]:
#Las cadenas tambien pueden ser instanciadas con un estado de "memoria"
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory #<--- golazo que veremos más adelante

In [7]:
conversation = ConversationChain(
    llm=chat,
    memory=ConversationBufferMemory()
)

In [8]:
conversation.run("Dime los primeros 5 números de la serie de fibonacci")

'¡Claro! Los primeros 5 números de la serie de Fibonacci son: 0, 1, 1, 2, 3. La serie de Fibonacci es una secuencia matemática en la que cada número es la suma de los dos anteriores.'

In [9]:
conversation.run("¿Y cuál es el séptimo?") #<-- preserva el contexto de la pregunta anterior

'El séptimo número en la serie de Fibonacci es 8.'